##Q1##

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
class SimpleNN:
    def __init__(self, input_size, hidden_size, output_size):
        np.random.seed(0)
        self.W1 = np.random.randn(input_size, hidden_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size)
        self.b2 = np.zeros((1, output_size))
        self.loss_history = []

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return (x > 0).astype(float)

    def forward(self, X):
        self.z1 = X @ self.W1 + self.b1
        self.a1 = self.relu(self.z1)
        self.z2 = self.a1 @ self.W2 + self.b2
        return self.z2

    def backward(self, X, y, output, lr=0.01):
        m = y.shape[0]
        dz2 = (output - y) * (2/m)
        dW2 = self.a1.T @ dz2
        db2 = np.sum(dz2, axis=0, keepdims=True)

        da1 = dz2 @ self.W2.T
        dz1 = da1 * self.relu_derivative(self.z1)
        dW1 = X.T @ dz1
        db1 = np.sum(dz1, axis=0, keepdims=True)

        self.W2 -= lr * dW2
        self.b2 -= lr * db2
        self.W1 -= lr * dW1
        self.b1 -= lr * db1

    def train(self, X, y, epochs=1000, lr=0.01):
        for epoch in range(epochs):
            output = self.forward(X)
            loss = np.mean((output - y)**2)
            self.loss_history.append(loss)
            self.backward(X, y, output, lr)

    def predict(self, X):
        return self.forward(X)

In [ ]:
def generate_3phase_data(frequency, points):
    t = np.linspace(0, 1/frequency, points).reshape(-1, 1)
    phase_a = np.sin(2*np.pi*frequency*t)
    phase_b = np.sin(2*np.pi*frequency*t + 2*np.pi/3)
    phase_c = np.sin(2*np.pi*frequency*t + 4*np.pi/3)
    output = np.hstack((phase_a, phase_b, phase_c))
    return t, output

In [ ]:
mse_list = []
best_mse = float('inf')
worst_mse = -float('inf')
best_pred, best_true = None, None
worst_pred, worst_true = None, None
best_nn, worst_nn = None, None

for points_per_cycle in range(1, 11):
    print(f"Training for {points_per_cycle} points per cycle...")

    t, y = generate_3phase_data(50, points_per_cycle)

    nn = SimpleNN(input_size=1, hidden_size=20, output_size=3)
    print("Training the neural network...")
    nn.train(t, y, epochs=3000, lr=0.0001)

    preds = nn.predict(t)
    mse = np.mean((preds - y)**2)
    mse_list.append(mse)

    print(f"MSE for {points_per_cycle} points: {mse}")

    if mse < best_mse:
        best_mse = mse
        best_pred, best_true = preds, y
        best_nn_t5 = nn
        print(f"New best MSE: {best_mse} at {points_per_cycle} points.")
    if mse > worst_mse:
        worst_mse = mse
        worst_pred, worst_true = preds, y
        worst_nn = nn
        print(f"New worst MSE: {worst_mse} at {points_per_cycle} points.")

print(f"\nTraining completed.")
print(f"Best MSE: {best_mse}")
print(f"Worst MSE: {worst_mse}")

Visualize Actual Data vs Predicted Data for Worst Case

In [ ]:
plt.figure(figsize=(20, 5))
plt.subplot(1, 2, 1)
plt.title('Best Case: Predicted vs Actual (Lowest MSE)')
plt.plot(best_true[:, 0], label='Phase A (True)', color='r')
plt.plot(best_pred[:, 0], label='Phase A (Predicted)', color='b')
plt.plot(best_true[:, 1], label='Phase B (True)', color='g')
plt.plot(best_pred[:, 1], label='Phase B (Predicted)', color='orange')
plt.plot(best_true[:, 2], label='Phase C (True)', color='b')
plt.plot(best_pred[:, 2], label='Phase C (Predicted)', color='purple')
plt.legend(loc='best')
plt.subplot(1, 2, 2)
plt.title('Worst Case: Predicted vs Actual (Highest MSE)')
plt.plot(worst_true[:, 0], label='Phase A (True)', color='r')
plt.plot(worst_pred[:, 0], label='Phase A (Predicted)', color='b')
plt.plot(worst_true[:, 1], label='Phase B (True)', color='g')
plt.plot(worst_pred[:, 1], label='Phase B (Predicted)', color='orange')
plt.plot(worst_true[:, 2], label='Phase C (True)', color='b')
plt.plot(worst_pred[:, 2], label='Phase C (Predicted)', color='purple')
plt.legend(loc='best')

plt.show()

In [ ]:
import seaborn as sns
plt.figure(figsize=(10,6))
plt.plot(range(1,11), mse_list, marker='o', color='blue')
plt.title('MSE vs Points Per Cycle (50Hz 3-Phase)', fontsize=14)
plt.xlabel('Points per Cycle', fontsize=12)
plt.ylabel('MSE', fontsize=12)
plt.grid(True)
plt.show()

print('Error Distribution')

error_best = (best_true - best_pred).flatten()
error_worst = (worst_true - worst_pred).flatten()

plt.figure(figsize=(12,5))
sns.histplot(error_best, bins=30, color='green', kde=True, label='Best Error', stat='density')
sns.histplot(error_worst, bins=30, color='red', kde=True, label='Worst Error', stat='density', alpha=0.6)
plt.title('Error Distribution (Best vs Worst Prediction)', fontsize=14)
plt.xlabel('Error Value')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()

print('Neural Network Weights Visualization')
fig, axs = plt.subplots(1, 2, figsize=(12,5))

sns.heatmap(best_nn_t5.W1, cmap='coolwarm', ax=axs[0])
axs[0].set_title('Best Model: W1 Weights')

sns.heatmap(best_nn_t5.W2, cmap='coolwarm', ax=axs[1])
axs[1].set_title('Best Model: W2 Weights')

plt.suptitle('Neural Network Weights Visualization', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(nn.loss_history, color='purple')
plt.title('Loss (MSE) During Training', fontsize=14)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.grid(True)
plt.show()

print('Hidden Layer Activations')
activations = nn.relu(t @ nn.W1 + nn.b1)

plt.figure(figsize=(10,6))
plt.imshow(activations, aspect='auto', cmap='viridis', interpolation='nearest')
plt.colorbar(label='Activation Strength')
plt.title('Hidden Layer Activations ', fontsize=14)
plt.xlabel('Neuron Index')
plt.ylabel('Sample Index')
plt.show()





In [ ]:
class SimpleNNVisualizer:
    def __init__(self, input_size, hidden_size, output_size):
        np.random.seed(0)
        self.W1 = np.random.randn(input_size, hidden_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size)
        self.b2 = np.zeros((1, output_size))

        self.activations = []
        self.gradients = []
        self.weights_history = []
        self.loss_history = []

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return (x > 0).astype(float)

    def forward(self, X):
        self.z1 = X @ self.W1 + self.b1
        self.a1 = self.relu(self.z1)
        self.z2 = self.a1 @ self.W2 + self.b2
        return self.z2

    def backward(self, X, y, output, lr=0.01):
        m = y.shape[0]
        dz2 = (output - y) * (2/m)
        dW2 = self.a1.T @ dz2
        db2 = np.sum(dz2, axis=0, keepdims=True)

        da1 = dz2 @ self.W2.T
        dz1 = da1 * self.relu_derivative(self.z1)
        dW1 = X.T @ dz1
        db1 = np.sum(dz1, axis=0, keepdims=True)

        # Gradients
        self.gradients.append({
            'dz2': dz2,
            'dW2': dW2,
            'dz1': dz1,
            'dW1': dW1,
        })

        self.W2 -= lr * dW2
        self.b2 -= lr * db2
        self.W1 -= lr * dW1
        self.b1 -= lr * db1

    def train(self, X, y, epochs=1000, lr=0.01, snapshot_interval=100):
        for epoch in range(epochs):
            output = self.forward(X)
            loss = np.mean((output - y) ** 2)
            self.loss_history.append(loss)

            if epoch % snapshot_interval == 0:
                self.activations.append((self.z1.copy(), self.a1.copy(), self.z2.copy()))
                self.weights_history.append((self.W1.copy(), self.W2.copy()))

            self.backward(X, y, output, lr)

    def predict(self, X):
        return self.forward(X)


Visualizing Working of the Simple Neural Network

In [ ]:
nn = SimpleNNVisualizer(input_size=1, hidden_size=10, output_size=3)
nn.train(t, y, epochs=3000, lr=0.01, snapshot_interval=100)

z1, a1, z2 = nn.activations[-1]

plt.figure(figsize=(12,6))
plt.subplot(2,1,1)
plt.imshow(z1, aspect='auto', cmap='plasma')
plt.colorbar()
plt.title('Hidden Pre-Activation z1 (before ReLU)')

plt.subplot(2,1,2)
plt.imshow(a1, aspect='auto', cmap='viridis')
plt.colorbar()
plt.title('Hidden Post-Activation a1 (after ReLU)')
plt.tight_layout()
plt.show()


In [ ]:
grad = nn.gradients[-1]

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.imshow(grad['dW1'], aspect='auto', cmap='coolwarm')
plt.colorbar()
plt.title('Gradient dW1 (Input -> Hidden)')

plt.subplot(1,2,2)
plt.imshow(grad['dW2'], aspect='auto', cmap='coolwarm')
plt.colorbar()
plt.title('Gradient dW2 (Hidden -> Output)')
plt.tight_layout()
plt.show()


In [ ]:
W1_start, W2_start = nn.weights_history[0]
W1_end, W2_end = nn.weights_history[-1]

plt.figure(figsize=(12,5))
plt.subplot(2,2,1)
plt.imshow(W1_start, cmap='bwr')
plt.title('W1 at Start')
plt.colorbar()

plt.subplot(2,2,2)
plt.imshow(W1_end, cmap='bwr')
plt.title('W1 at End')
plt.colorbar()

plt.subplot(2,2,3)
plt.imshow(W2_start, cmap='bwr')
plt.title('W2 at Start')
plt.colorbar()

plt.subplot(2,2,4)
plt.imshow(W2_end, cmap='bwr')
plt.title('W2 at End')
plt.colorbar()

plt.tight_layout()
plt.show()


##Q2##

In [ ]:
def generate_sine_wave_data(frequency, points):
    t = np.linspace(0, 1/frequency, points).reshape(-1, 1)
    y = np.sin(2 * np.pi * frequency * t)
    return t, y

In [ ]:
mse_list = []
loss_history_all_models = {}

t, y = generate_sine_wave_data(130, 1000)

for hidden_neurons in range(1, 201, 10):
    print(f"Training model with {hidden_neurons} hidden neurons...")

    nn = SimpleNN(input_size=1, hidden_size=hidden_neurons, output_size=1)
    nn.train(t, y, epochs=3000, lr=0.0001)
    preds = nn.predict(t)

    mse = np.mean((preds - y)**2)
    mse_list.append(mse)
    loss_history_all_models[hidden_neurons] = nn.loss_history

    print(f"MSE for {hidden_neurons} hidden neurons: {mse}")
plt.figure(figsize=(8, 6))
plt.plot(t, y, label="True Sine Wave (130Hz)")
plt.title('True 130 Hz Single-Phase Sine Wave')
plt.xlabel('Time (t)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.legend()
plt.show()

plt.figure(figsize=(8, 6))
for hidden_neurons, loss_history in loss_history_all_models.items():
    plt.plot(range(0, len(loss_history)*100, 100), loss_history, label=f'{hidden_neurons} Neurons')
plt.title('Training Loss vs. Epochs for Different Hidden Layer Sizes')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 6))
plt.plot(range(1, 201, 10), mse_list, marker='o')
plt.title('MSE vs Number of Neurons in Hidden Layer (130Hz Sine Wave)')
plt.xlabel('Number of Neurons in Hidden Layer')
plt.ylabel('MSE (Error)')
plt.grid(True)
plt.show()

best_mse = min(mse_list)
best_neurons = (mse_list.index(best_mse) + 1) * 10
print(f"Best model has {best_neurons} neurons with MSE: {best_mse}")

best_nn = SimpleNN(input_size=1, hidden_size=best_neurons, output_size=1)
best_nn.train(t, y, epochs=3000, lr=0.0001)
best_pred = best_nn.predict(t)

plt.figure(figsize=(8, 6))
plt.plot(t, y, label="True Sine Wave")
plt.plot(t, best_pred, label="Best Model Prediction", linestyle='--')
plt.title(f'Best Model Prediction vs True Output ({best_neurons} Neurons)')
plt.xlabel('Time (t)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)
plt.show()

worst_mse_index = mse_list.index(max(mse_list))
worst_neurons = (worst_mse_index + 1) * 10
worst_nn = SimpleNN(input_size=1, hidden_size=worst_neurons, output_size=1)
worst_nn.train(t, y, epochs=3000, lr=0.0001)
worst_pred = worst_nn.predict(t)

plt.figure(figsize=(8, 6))
plt.plot(t, y, label="True Sine Wave")
plt.plot(t, worst_pred, label="Worst Model Prediction", linestyle='--')
plt.title(f'Worst Model Prediction vs True Output ({worst_neurons} Neurons)')
plt.xlabel('Time (t)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)
plt.show()

##Q3##

In [ ]:

def generate_sine_wave_data(frequency, points):
    t = np.linspace(0, 1/frequency, points).reshape(-1, 1)
    y = np.sin(2 * np.pi * frequency * t)
    return t, y

t_130, y_130 = generate_sine_wave_data(130, 250)

hidden_sizes = [ 10, 20,30]
learning_rates = [1e-2, 1e-3, 1e-4]
epoch_list = [1000,2000, 3000,4000]

best_new_mse = float('inf')
best_new_config = None
best_new_model = None

new_model_results = []

print("\n--- Tuning New Model ---")

for hidden_size in hidden_sizes:
    for lr in learning_rates:
        for epochs in epoch_list:
            model = SimpleNN(input_size=1, hidden_size=hidden_size, output_size=3)
            model.train(t_130, y_130, epochs=epochs, lr=lr)
            preds = model.predict(t_130)
            mse = np.mean((preds - y_130)**2)

            print(f"Hidden Size: {hidden_size}, LR: {lr}, Epochs: {epochs} -> MSE: {mse:.6f}")

            new_model_results.append((hidden_size, lr, epochs, mse))

            if mse < best_new_mse:
                best_new_mse = mse
                best_new_config = (hidden_size, lr, epochs)
                best_new_model = model

print(f"\nBest New Model Config: Hidden Size={best_new_config[0]}, LR={best_new_config[1]}, Epochs={best_new_config[2]}, MSE={best_new_mse:.6f}")

In [ ]:
import copy
pretrained_nn=best_nn_t5
best_pretrain_mse = float('inf')
best_pretrain_config = None
best_pretrained_model = None

pretrain_model_results = []

print("\n--- Tuning Pretrained Model ---")

for hidden_size in hidden_sizes:
    for lr in learning_rates:
        for epochs in epoch_list:
            model = copy.deepcopy(pretrained_nn)
            model.train(t_130, y_130, epochs=epochs, lr=lr)
            preds = model.predict(t_130)
            mse = np.mean((preds - y_130)**2)

            print(f"Hidden Size: {hidden_size}, LR: {lr}, Epochs: {epochs} -> MSE: {mse:.6f}")

            pretrain_model_results.append((hidden_size, lr, epochs, mse))

            if mse < best_pretrain_mse:
                best_pretrain_mse = mse
                best_pretrain_config = (hidden_size, lr, epochs)
                best_pretrained_model = model

print(f"\nBest Fine-tuned Pretrained Model Config: Hidden Size={best_pretrain_config[0]}, LR={best_pretrain_config[1]}, Epochs={best_pretrain_config[2]}, MSE={best_pretrain_mse:.6f}")


In [ ]:
new_model_results = np.array(new_model_results)
pretrain_model_results = np.array(pretrain_model_results)

plt.figure(figsize=(20,18))
for lr in np.unique(new_model_results[:,1]):
    for epochs in np.unique(new_model_results[:,2]):
        mask = (new_model_results[:,1] == lr) & (new_model_results[:,2] == epochs)
        plt.plot(new_model_results[mask][:,0], new_model_results[mask][:,3], marker='o', label=f"LR={lr}, Epochs={epochs}")
plt.title('New Model: Hidden Size vs MSE')
plt.xlabel('Hidden Size')
plt.ylabel('MSE')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()
print('\n')
plt.figure(figsize=(20,18))
for lr in np.unique(pretrain_model_results[:,1]):
    for epochs in np.unique(pretrain_model_results[:,2]):
        mask = (pretrain_model_results[:,1] == lr) & (pretrain_model_results[:,2] == epochs)
        plt.plot(pretrain_model_results[mask][:,0], pretrain_model_results[mask][:,3], marker='o', label=f"LR={lr}, Epochs={epochs}")
plt.title('Pretrained Model: Hidden Size vs MSE')
plt.xlabel('Hidden Size')
plt.ylabel('MSE')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
new_preds = best_new_model.predict(t_130)
pretrain_preds = best_pretrained_model.predict(t_130)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(t_130, y_130, label="True 130Hz Sine")
plt.plot(t_130, new_preds, '--', label="Best New Model Prediction")
plt.title(f'Best New Model (MSE: {best_new_mse:.6f})')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)
plt.show()
print('\n')
plt.figure(figsize=(10,5))
plt.plot(t_130, y_130, label="True 130Hz Sine")
plt.plot(t_130, pretrain_preds, '--', label="Best Fine-Tuned Pretrained Model Prediction")
plt.title(f'Best Fine-Tuned Pretrained Model (MSE: {best_pretrain_mse:.6f})')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)
plt.show()

print('\n')

print("\n--- Final Comparison ---")
print(f"Best New Model MSE: {best_new_mse:.6f}")
print(f"Best Fine-tuned Pretrained Model MSE: {best_pretrain_mse:.6f}")

if best_pretrain_mse < best_new_mse:
    print("Fine-tuned pretrained model performs BETTER than training new model from scratch!")
else:
    print("Training new model from scratch performs BETTER than fine-tuning pretrained model!")

##Q4##

In [ ]:
def generate_waveform(sample_points=100, freq_range=(1,20)):
    freq = np.random.uniform(*freq_range)
    t = np.linspace(0, 1, sample_points)
    y = np.sin(2 * np.pi * freq * t)
    return t, y, freq

X = []
y = []
n_samples = 5000
sample_points = 100

for _ in range(n_samples):
    _, waveform, freq = generate_waveform(sample_points=sample_points)
    X.append(waveform)
    y.append(freq)

X = np.array(X)
y = np.array(y)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = y_train.reshape(-1,1)
y_val = y_val.reshape(-1,1)


In [ ]:
hidden_sizes = [8, 16, 32, 64]
learning_rates = [1e-2, 1e-3, 1e-4]
epoch_list = [1000, 2000, 3000]

results = []

best_mse = float('inf')
best_config = None
best_model = None

for hidden_size in hidden_sizes:
    for lr in learning_rates:
        for epochs in epoch_list:
            print(f"Training with Hidden Size={hidden_size}, LR={lr}, Epochs={epochs}")
            model = SimpleNN(input_size=100, hidden_size=hidden_size, output_size=1)
            model.train(X_train, y_train, epochs=epochs, lr=lr)
            preds = model.predict(X_train)
            mse = np.mean((preds - y_train)**2)

            print(f"-> MSE: {mse:.6f}\n")

            results.append((hidden_size, lr, epochs, mse))

            if mse < best_mse:
                best_mse = mse
                best_config = (hidden_size, lr, epochs)
                best_model = model

print(f"\nBest Config: Hidden Size={best_config[0]}, LR={best_config[1]}, Epochs={best_config[2]}, Best MSE={best_mse:.6f}")

In [ ]:
import pandas as pd
results_df = pd.DataFrame(results, columns=["hidden_size", "lr", "epochs", "mse"])
best_epoch = 3000
filtered_df = results_df[results_df['epochs'] == best_epoch]

pivot_table = filtered_df.pivot(index="hidden_size", columns="lr", values="mse")

plt.figure(figsize=(10,6))
sns.heatmap(pivot_table, annot=True, fmt=".6f", cmap="coolwarm")
plt.title(f"Validation MSE Heatmap (Epochs={best_epoch})")
plt.ylabel("Hidden Size")
plt.xlabel("Learning Rate")
plt.show()


In [ ]:
best_lr = best_config[1]

filtered_df2 = results_df[(results_df['lr'] == best_lr) & (results_df['epochs'] == best_epoch)]

plt.figure(figsize=(8,5))
plt.plot(filtered_df2['hidden_size'], filtered_df2['mse'], marker='o')
plt.title(f"Hidden Size vs MSE (LR={best_lr}, Epochs={best_epoch})")
plt.xlabel("Hidden Layer Size")
plt.ylabel("MSE")
plt.grid()
plt.show()


In [ ]:
model=best_model

In [ ]:
history=model.loss_history
plt.figure(figsize=(8,5))
plt.plot(history)
plt.xlabel('Epochs')
plt.ylabel('MSE Loss')
plt.title('Training Loss over Epochs')
plt.grid()
plt.show()


In [ ]:
y_pred = model.predict(X_val)

plt.figure(figsize=(8,5))
plt.scatter(y_val, y_pred, alpha=0.5)
plt.plot([1,20],[1,20],'r--')
plt.xlabel("True Frequency (Hz)")
plt.ylabel("Predicted Frequency (Hz)")
plt.title("Predicted vs True Frequency (Validation Set)")
plt.grid()
plt.show()

val_mse = np.mean((y_pred - y_val)**2)
print(f"Validation MSE: {val_mse:.6f}")


In [ ]:
def find_dominant_frequency_fft(signal, sample_rate=100):
    n = len(signal)
    freqs = np.fft.rfftfreq(n, d=1/sample_rate)
    fft_vals = np.abs(np.fft.rfft(signal))
    dominant_freq = freqs[np.argmax(fft_vals)]
    return dominant_freq


In [ ]:
sample_points = 100
sample_rate = sample_points
n_tests = 20
errors_nn = []
errors_fft = []

for _ in range(n_tests):
    t, waveform, true_freq = generate_waveform(sample_points=sample_points)
    waveform_input = waveform.reshape(1, -1)
    nn_pred = model.predict(waveform_input).flatten()[0]

    fft_pred = find_dominant_frequency_fft(waveform, sample_rate)

    error_nn = abs(nn_pred - true_freq)
    error_fft = abs(fft_pred - true_freq)

    errors_nn.append(error_nn)
    errors_fft.append(error_fft)

    print(f"True Frequency: {true_freq:.2f} Hz | NN Pred: {nn_pred:.2f} Hz | FFT Pred: {fft_pred:.2f} Hz")

print(f"\nAverage Absolute Error (NN): {np.mean(errors_nn):.4f} Hz")
print(f"Average Absolute Error (FFT): {np.mean(errors_fft):.4f} Hz")


In [ ]:
plt.figure(figsize=(8,5))
plt.plot(errors_nn, label='NN Error', marker='o')
plt.plot(errors_fft, label='FFT Error', marker='x')
plt.xlabel('Test Sample Index')
plt.ylabel('Absolute Frequency Error (Hz)')
plt.title('NN vs FFT Frequency Estimation Error')
plt.legend()
plt.grid()
plt.show()
